<a href="https://colab.research.google.com/github/rajathpatel23/Chord-implementation/blob/master/InceptionNet_freeze_224.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import numpy as np
import pandas as pd
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from pickle import dump
from keras.layers import Dense, Flatten, GlobalAveragePooling2D
from keras.regularizers import l2
from sklearn.externals import joblib
import cv2
from keras.models import load_model

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import tensorflow as tf
import os
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(False)
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))
try:
 device_name = os.environ['COLAB_TPU_ADDR']
 TPU_ADDRESS = 'grpc://' + device_name
 print('Found TPU at: {}'.format(TPU_ADDRESS))
except KeyError:
 print('TPU not found')

TPU not found


In [0]:
def get_tuning(X_in):
      tuned_out = []
      for x in range(len(X_in)):
            if X_in[x][0] >= 0.5:
                  tuned_out.append(1)
            else:
                  tuned_out.append(0)
      return tuned_out

def get_confusion_matrix(y_t, y_pred):
      tn, fp, fn, tp = confusion_matrix(y_t, y_pred).ravel()
      sensitivity = tp/(tp + fn)
      specificity = tn/(tn + fp)
      PPV = tp/(tp + fp)
      NPV = tn/(tn + fn)
      return sensitivity, specificity, PPV, NPV

def get_aucpr(y_t, y_pred):
      precision, recall, thres  = precision_recall_curve(y_t, y_pred)
      aucpr = auc(recall, precision)
      return aucpr


def get_processed_X(X):
      X_copy  = X
      for arr in range(len(X_copy)):
            a = np.min(X_copy[arr])
            b = np.max(X_copy[arr])
            X_copy[arr] = np.divide(np.subtract(X_copy[arr], a), (b-a))
      return X_copy 
#%%
def get_reshape(X):
      X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
      return X

In [0]:
train_dict_1 = joblib.load('/content/drive/My Drive/Dataset/train_stft_data_1.pkl')
train_dict_2 = joblib.load('/content/drive/My Drive/Dataset/train_stft_data_2.pkl')
X = np.concatenate((train_dict_1["stft"], train_dict_2["stft"]))
y = np.concatenate((train_dict_1["MT"], train_dict_2["MT"]))
del train_dict_1
del train_dict_2

X_new = [0 for x in range(len(X))]
dim = (299, 299)
i = 0
for elem in X:
  elem_new = cv2.resize(elem, dim, interpolation = cv2.INTER_AREA)
#   print(elem_new.shape)
  X_new[i] = (elem_new)  
  i+=1

del X
X_new = np.array(X_new)
X_new = get_processed_X(X_new)
X_new = np.stack((X_new,)*3, axis=-1)
print(X_new.shape)



# X = get_processed_X(X)
# X = np.stack((X,)*3, axis=-1)
# print(X.shape)

(7683, 299, 299, 3)


In [0]:
# def train_input_fn(X, y, batch_size=64):
#     # Convert the inputs to a Dataset.
#     dataset = tf.data.Dataset.from_tensor_slices((X,y))
# # Shuffle, repeat, and batch the examples.
#     dataset = dataset.cache()
#     dataset = dataset.shuffle(1000, reshuffle_each_iteration=True)
#     dataset = dataset.repeat()
#     dataset = dataset.batch(batch_size, drop_remainder=True)
# # Return the dataset.
#     return dataset
# dataset_train = train_input_fn(X_new, y, batch_size=10)

ValueError: ignored

In [0]:
model_1 = InceptionV3(input_shape=(299, 299, 3))
# print(model_1.summary())









96116736/96112376 [==============================] - 1s 0us/step


In [0]:
model_1.layers.pop()
for layer in model_1.layers:
  if layer.trainable == True:
    layer.trainable = False

In [12]:

# average_layer =GlobalAveragePooling2D()(model_1.output)
# # flat1 =Flatten()(average_layer)
# class1 = Dense(1024, activation='relu')(average_layer)
# class2 = Dense(512, activation='relu')(class1)
output = Dense(1, activation='sigmoid', activity_regularizer=l2(0.01))(model_1.layers[-1].output)
model = Model(inputs = model_1.inputs, outputs = output)
model.summary()
learning_rate = 1e-3
opt = tf.train.AdamOptimizer(learning_rate)
model.compile(loss='binary_crossentropy',optimizer=opt)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [0]:
# tpu_model = tf.contrib.tpu.keras_to_tpu_model(
#     model,
#     strategy=tf.contrib.tpu.TPUDistributionStrategy(
#         tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)))

In [10]:
from sklearn.utils import class_weight
from keras.callbacks import ModelCheckpoint, EarlyStopping

sample_weights = class_weight.compute_sample_weight(class_weight="balanced", y=y)
callbacks = [EarlyStopping(monitor='loss', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=True), 
             ModelCheckpoint(filepath='/content/drive/My Drive/Dataset/Inceptionnet_model_new/Inception_model_freeze_model.h5', monitor='loss', save_best_only=True)]
history = model.fit(X_new, y,epochs=30,batch_size=64,shuffle=True, verbose=1, 
                    sample_weight=sample_weights, callbacks=callbacks)

Epoch 1/30
7683/7683 [==============================] - 82s 11ms/step - loss: 0.7357


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:135: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


Epoch 2/30
7683/7683 [==============================] - 73s 10ms/step - loss: 0.6666
Epoch 3/30
7683/7683 [==============================] - 73s 10ms/step - loss: 0.6344
Epoch 4/30
7683/7683 [==============================] - 74s 10ms/step - loss: 0.6195
Epoch 5/30
7683/7683 [==============================] - 73s 9ms/step - loss: 0.6376
Epoch 6/30
7683/7683 [==============================] - 74s 10ms/step - loss: 0.5859
Epoch 7/30
7683/7683 [==============================] - 73s 9ms/step - loss: 0.6181
Epoch 8/30
7683/7683 [==============================] - 74s 10ms/step - loss: 0.5725
Epoch 9/30
7683/7683 [==============================] - 74s 10ms/step - loss: 0.5755
Epoch 10/30
7683/7683 [==============================] - 74s 10ms/step - loss: 0.5597
Epoch 11/30
7683/7683 [==============================] - 74s 10ms/step - loss: 0.5830
Epoch 12/30
7683/7683 [==============================] - 75s 10ms/step - loss: 0.5980
Epoch 13/30
7683/7683 [==============================] - 75s 10m

In [7]:
#%%
model = load_model('/content/drive/My Drive/Dataset/Inceptionnet_model_new/Inception_model_freeze_model.h5')
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, cohen_kappa_score, auc, roc_curve
from sklearn.metrics import average_precision_score, confusion_matrix, precision_recall_curve, classification_report
def performance_analysis(y_2, y_test_1):      
      tuned_y_2 = get_tuning(y_2)
      roc_auc_test = roc_auc_score(y_test_1, y_2)
      print("The test roc is : ", roc_auc_test)
      aucpr_test = get_aucpr(y_test_1, y_2)
      print("The test aucpr is: ", aucpr_test)
      sense_test_, specs_test_, PPV_test_, NPV_test_ = get_confusion_matrix(y_test_1, tuned_y_2)
      print("The test sensitivity is: ", sense_test_)
      print("The test specificity is: ", specs_test_)
      print("The test PPV is: ", PPV_test_)
      print("The test NPV is: ", NPV_test_)
      return True

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [0]:
test_1 = joblib.load('/content/drive/My Drive/Dataset/test_stft_data_1.pkl')
test_2 = joblib.load('/content/drive/My Drive/Dataset/test_stft_data_2.pkl')

In [9]:
X = np.concatenate((test_1["stft"], test_2["stft"]))
y_test = np.concatenate((test_1["MT"], test_2["MT"]))
del test_1
del test_2
X_new = [0 for x in range(len(X))]
dim = (299, 299)
i = 0
for elem in X:
  elem_new = cv2.resize(elem, dim, interpolation = cv2.INTER_AREA)
#   print(elem_new.shape)
  X_new[i] = (elem_new)  
  i+=1

del X
X_new = np.array(X_new)
X_new = get_processed_X(X_new)
X_new = np.stack((X_new,)*3, axis=-1)
print(X_new.shape)


(6831, 299, 299, 3)


In [11]:
y_pred_MT = model.predict(X_new)
performance_analysis(y_pred_MT, y_test)

The test roc is :  0.8209961954627049
The test aucpr is:  0.08450107074364255
The test sensitivity is:  0.17391304347826086
The test specificity is:  0.9716121320782908
The test PPV is:  0.11214953271028037
The test NPV is:  0.9827716487834366


True

In [0]:
model.save('/content/drive/My Drive/Dataset/Inception_30_32_MT_STFT.h5')

In [0]:
y_pred_MT.shape